In [9]:
from main import generate_bundle


ImportError: cannot import name 'generate_bundle' from 'main' (/Users/christopherchandler/code_repos/christopher-chandler/Python/nlp/rub/bundled_gap_filling/main.py)